Equation 1:

\begin{align*}
0 &= \max_{e}\min_{h_2} b\left[\delta \eta \log e - \tau z_2 e {\color{red}{+}} \xi_m \frac{(h_2)^2}{2}\right] - \ell e {\color{red}{-}} \frac{\partial \psi}{\partial b}(b,z_2;\ell) \delta b\\
&+\left[\frac{\partial \psi}{\partial z_2}(b,z_2;\ell)\right]\left[-\rho(z_2-\mu_2)+\sqrt{z_2}\sigma_2 h_2\right] + \left[\frac{\partial^2 \psi}{\partial(z_2)^2}(b,z_2;\ell)\right]\left(\frac{z_2|\sigma_2|^2}{2}\right)
\end{align*}

Equation 2:

\begin{align*}
0 &= \max_{e}\min_{h_2} -\delta \phi(y, z_2) + \delta \eta \log e -\delta(1-\eta)y z_2 {\color{red}{+}} \xi_m \frac{(h_2)^2}{2} {\color{red}{-}} \frac{\partial \phi}{\partial y}(y,z_2) e\\
&+\left[\frac{\partial \phi}{\partial z_2}(y,z_2)\right]\left[-\rho(z_2-\mu_2)+\sqrt{z_2}\sigma_2 h_2\right] + \left[\frac{\partial^2 \phi}{\partial(z_2)^2}(y,z_2)\right]\left(\frac{z_2|\sigma_2|^2}{2}\right)
\end{align*}